In [1]:
import torch

/home/dan/anaconda3/envs/torch-cuda11/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
shapes = torch.as_tensor([(6, 4), (3, 2)], dtype=torch.long).cuda()

In [4]:
shapes

tensor([[6, 4],
        [3, 2]], device='cuda:0')

In [3]:
level_start_index = torch.cat((shapes.new_zeros((1, )), (shapes[:,0]*shapes[:,1]).cumsum(0)[:-1]))

In [5]:
shapes.new_zeros((1, ))

tensor([0], device='cuda:0')

In [13]:
(shapes[:,0]*shapes[:,1]).cumsum(0)[:-1]

tensor([24], device='cuda:0')